In [1]:
import yfinance as yf
from datetime import date, datetime
import pandas as pd
import sqlite3
import xlwings as xw
import time, os, sys

This is just to test different methods before implementation into main python file. 

1st part are the tested methods

In [1]:
# ticker = "D05.SI"
# dbs = yf.Ticker(ticker)
# cur_price = dbs.fast_info
# print(cur_price.last_price, cur_price.previous_close)

#Get Currency rates from yfinance
# cur = yf.Ticker("HKDSGD=X")
# a = cur.fast_info
# print(f"{a.last_price:.6f}") #Prints conversion rate to 6th decimal place

# ticker = "D05.SI"
# dbs = yf.Ticker(ticker)
# a = dbs.dividends # Pandas series object

# #Clean data
# df = pd.DataFrame(a).reset_index()
# #print(df.dtypes) {Date = datetime64[ns, Asia/Singapore], Dividends = float64}
# #Date is Ex date, Dividend currency has to be changed with Forex Rates

# #Find all dividend after first buy date
# dfCleaned = df[~(df['Date'] < '2020-06-04')] #Date gotten from 1st Buy Transaction of Excel
# #print(dfCleaned)

#Main function to pull dividend data from yfinance, clean it by removing all data after a certain date, and appending it to my sqlite db
# tickers = ['D05.SI', 'O39.SI', 'RW0U.SI', '3988.HK']

'''
for ticker in tickers:
    try:
        stock = yf.Ticker(ticker)
        dividend_data = stock.dividends
        df = pd.DataFrame(dividend_data).reset_index()
        
        dfCleaned = df[~(df['Date'] <= '2020-06-04')] #Get date from existing db
        dfCleaned.insert(0, 'Ticker', ticker)
        
        #Transfer pandas df into sqlite db
        dfCleaned.to_sql('dividends', conn, if_exists='append', index=False)
        conn.commit()
        
    except Exception as error:
        print('Something went wrong')
        print(error)

if not dfCleaned.empty:
    print('It exists:', dfCleaned)
else:
    print('huh?', dfCleaned)
'''


#Create the table for dividend and commit transaction
# c.execute('DROP TABLE dividends')
# c.execute('CREATE TABLE dividends (ticker text, date text, dividends float)') #Add if not exist for main script
# conn.commit()

'''
#Get latest date of dividend for each stock and save to a dict
c.execute("SELECT ticker, MAX(date(date, 'localtime')) FROM dividends group by ticker")
results = c.fetchall()
list = []
for result in results:
    list.append(result)
    
tmp_dict = dict(list)
result_dict = {}
#Clean the date field, by keeping only yyyy-mm-dd, and save into another dict
for key, item in tmp_dict.items():
    item = item.split(" ", 1)[0]
    result_dict[key] = item
    
print(result_dict)
'''

'''
tickers = ['D05.SI', 'O39.SI', 'RW0U.SI', '3988.HK']

for ticker in tickers:
    try:
        stock = yf.Ticker(ticker)
        dividend_data = stock.dividends
        df = pd.DataFrame(dividend_data).reset_index()
            
        #Check if there is any data in the db
        if result_dict:
            dfCleaned = df[~(df['Date'] <= result_dict[ticker])] #Get date from existing db
        #If db is empty, use a default date to pull all dividends from that date onwards
        else:
            dfCleaned = df[~(df['Date'] <= '2020-06-04')] #Just set a date, in yyyy-mm-dd

        #Check if dfClean is empty or not, skips appending df to sqlite3 db if so
        if not dfCleaned.empty:
            #Insert a column to keep track of stock's ticker symbol
            dfCleaned.insert(0, 'Ticker', ticker)
                
            #Transfer pandas df into sqlite db
            dfCleaned.to_sql('dividends', conn, if_exists='append', index=False)
            conn.commit()
        
    except Exception as error:
        print('Something went wrong')
        print(error)
'''

'''
#Get First buy date test
def main():
    wb = xw.Book.caller()
    buyTransac = wb.sheets('Buy_Transactions')
    test = buyTransac.range("A3:C3").options(expand='down').value
    test_dict = {}
    for t in test:
        if t[2] in test_dict:
            pass
        else:
            test_dict[t[2]] = t[0].strftime("%Y-%m-%d") #yyyy-mm-dd
    print(test_dict)

if __name__ == "__main__":
    xw.Book("dividendTracker.xlsm").set_mock_caller()
    main()
'''

'''
#Tried selenium and bs4 for scraping, but does not work in this case as its too slow
#Change options for selenium
chrome_options = Options()
chrome_options.add_argument("--headless")

#Make webdriver use Chrome Browser
driver = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver", options = chrome_options)

dividends = []
driver.get("http://www.aastocks.com/en/stocks/analysis/company-fundamental/dividend-history?symbol=01830")

content = driver.page_source
soup = BeautifulSoup(content)
a = soup.findAll('table', attrs={'class': 'cnhk-cf tblM s4 s5 mar15T'})
rows = a[1].findAll('tr') #All rows of the table

# row = rows[1].findAll('td')

# for element in b:
#     print(element.text)

for row in rows:
    t = row.findAll('td')
    for ele in t:
        print(ele.text)
'''

'''
dfs = pd.read_html('http://www.aastocks.com/en/stocks/analysis/company-fundamental/dividend-history?symbol=01830')
dividend_date_for_perfectshape = dfs[25] #Pandas df
dividend_date_for_perfectshape.drop(columns = [0, 1, 2, 4, 6, 7], inplace = True)
dividend_date_for_perfectshape.rename(columns = {3: 'dividends', 5: 'date'}, inplace = True)
dividend_date_for_perfectshape = dividend_date_for_perfectshape[~(dividend_date_for_perfectshape['date'] <= '2020-06-04')]
dividend_date_for_perfectshape
dividend_date_for_perfectshape["dividends"] = dividend_date_for_perfectshape["dividends"].str.split("HKD ").str[1]
dividend_date_for_perfectshape
'''

'''
dfs = pd.read_html('http://www.aastocks.com/en/stocks/analysis/dividend.aspx?symbol=03988')
dividend_date_for_boc = dfs[25] #Pandas df
dividend_date_for_boc.drop(columns = [0, 1, 2, 4, 6, 7], inplace = True)
dividend_date_for_boc.rename(columns = {3: 'dividends', 5: 'date'}, inplace = True)
dividend_date_for_boc = dividend_date_for_boc[~(dividend_date_for_boc['date'] <= '2020-06-04')]
dividend_date_for_boc
# dividend_date_for_boc["dividends"] = dividend_date_for_boc["dividends"].str.split("HKD ").str[1]
# dividend_date_for_boc["dividends"] = dividend_date_for_boc["dividends"].str.replace(")", "", regex = False)
# dividend_date_for_boc

#Clean Dividends column
divs = uncleanedDf.loc[:, ['Dividends']]
divs = divs['Dividends'].str.split("HKD ").str[1]
divs = divs.str.replace(")", "", regex = False)
                
#Clean Date column
dates = uncleanedDf.loc[:, ['Date']]
dates = dates['Date'].str.replace("/", "-", regex = False)

cleanedDF = pd.concat([dates, a], axis = 1)
cleanedDF
'''

'''
def get_and_clean_info_from_source(ticker):
    try:     
        #Check ticker symbol, and returns a df with columns = ['date', 'dividends']
        if ticker == "1830.HK" or ticker == "3988.HK": 
            if ticker == "1830.HK": #Perfect Shape Medical Stock Ticker
                dfs = pd.read_html('http://www.aastocks.com/en/stocks/analysis/company-fundamental/dividend-history?symbol=01830')
            elif ticker == "3988.HK": #Bank of China Stock Ticker
                dfs = pd.read_html('http://www.aastocks.com/en/stocks/analysis/dividend.aspx?symbol=03988')
                
            uncleanedDf = dfs[25] #26th df in the webpage
            
            #Clean the df
            uncleanedDf.drop(columns = [0, 1, 2, 4, 6, 7], inplace = True)
            uncleanedDf.rename(columns = {3: 'Dividends', 5: 'Date'}, inplace = True)
            
            #3988.HK requires additional cleaning, resulting in another if statement
            if ticker == "1830.HK":
                #Clean Dividends column
                divs = uncleanedDf.loc[:, ['Dividends']]
                divs = divs["Dividends"].str.split("HKD ").str[1]

                #Clean Date column
                dates = uncleanedDf.loc[:, ['Date']]
                dates = dates['Date'].str.replace("/", "-", regex = False)
            elif ticker == "3988.HK":
                #Clean Dividends column
                divs = uncleanedDf.loc[:, ['Dividends']]
                divs = divs['Dividends'].str.split("HKD ").str[1]
                divs = divs.str.replace(")", "", regex = False)
                
                #Clean Date column
                dates = uncleanedDf.loc[:, ['Date']]
                dates = dates['Date'].str.replace("/", "-", regex = False)
            
            divs = divs.astype('float64')
            dates = pd.to_datetime(dates, format= '%Y-%m-%d', errors= 'coerce').dt.tz_localize('UTC').dt.tz_convert('Asia/Singapore')
            #Concat back both columns into a single df
            df = pd.concat([dates, divs], axis = 1) 
            #Drop NaN or NaT values
            df = df.dropna()
        else:
            stock = yf.Ticker(ticker)
            dividend_data = stock.dividends
            df = pd.DataFrame(dividend_data).reset_index()
        
        return df
    except Exception as error:
        print(error)
        
df1 = get_and_clean_info_from_source("D05.SI")
df2 = get_and_clean_info_from_source("3988.HK")
print(df1.dtypes)
print(df2)
print(df2.dtypes)
'''

'''
#Check date of dividends against dates of buy transactions, figure out total amount of stock held at the point in time 

def main():
    wb = xw.Book.caller()
    buyTransac = wb.sheets('Buy_Transactions')
    test = buyTransac.range("A3:K3").options(expand='down').value #Nested list
    test_list = []
    
    for row in test:
        test_list.append([row[2], row[0], row[10]])
        
    checkDF = pd.DataFrame(test_list).reset_index(drop= True)
    checkDF.rename(columns = {0: "Ticker", 1: "Date", 2: "Amount_of_shares"}, inplace=True)

    tickers = ["D05.SI", "O39.SI", "M44U.SI", "H78.SI"]
    
    for ticker in tickers:
        stock = yf.Ticker(ticker)
        dividend_data = stock.dividends
        df = pd.DataFrame(dividend_data).reset_index()
        dfCleaned = df[~(df['Date'] <= "2021-04-29")].reset_index(drop= True)
        
        #Insert required rows into df
        dfCleaned.insert(0, 'Ticker', ticker)
        dfCleaned.insert(3, 'Amount_of_shares', 0)
        
        #Change Dates into same datetime aware format for comparison
        try:
            dfCleaned["Date"].dt.tz_convert("Asia/Singapore")
            checkDF.loc[:, "Date"] = checkDF["Date"].dt.tz_localize("Asia/Singapore")
        except Exception as error:
            print(error)
        
        for divIndex, divSeries in dfCleaned.iterrows():
            for boughtIndex, boughtSeries in checkDF[checkDF["Ticker"] == ticker].sort_values(by=["Date"], ascending=False).iterrows():
                if divSeries["Date"] > boughtSeries["Date"]:
                    dfCleaned.at[divIndex, "Amount_of_shares"] = boughtSeries["Amount_of_shares"]
                    break
                
        print(dfCleaned)

if __name__ == "__main__":
    xw.Book("dividendTracker.xlsm").set_mock_caller()
    main()
'''

'''
# Test - Get Stock table rows from main sheet without needing to change number everytime a new stock is added
def main():
    wb = xw.Book.caller()
    mainSheet = wb.sheets('Portfolio')
    last_col = mainSheet.range("B8").end('right').column
    last_row = mainSheet.range("B8").end('down').row
    targetRange = mainSheet.range((8,2), (last_row, last_col)).rows
    
    x = 0

    for row in targetRange:
        print(row)
        x += 1
        print(x)
    

if __name__ == "__main__":
    xw.Book("dividendTracker.xlsm").set_mock_caller()
    main()
'''

'''
# Test to get pandas df ready for plotting
def main():
    wb = xw.Book.caller()
    mainSheet = wb.sheets('Portfolio')
    graphs = wb.sheets('Graphs')
    
    cost = mainSheet.range("A1:B4").options(convert=pd.DataFrame).value.reset_index()
    cost = cost.astype({'Cost': 'float64'})
    print(cost.dtypes)
    print(cost)
    print(cost["index"])
    print(cost["Cost"])
    

if __name__ == "__main__":
    xw.Book("dividendTracker.xlsm").set_mock_caller()
    main()
'''

# Testing Area

In [5]:
#Initialize the sqlite3 connection and table

#Connect to sqlite db with absolute path
#Raw string used due to SyntaxError (Unicode error)
conn = sqlite3.connect(r"C:\Users\Zhen Xuan\OneDrive\Desktop\CodingStuff\PersonalProjects\AutomatedDividendTracker\dividend_record.db")
#Create cursor
c = conn.cursor()

In [6]:

#Initialize table if not created, else ignore
c.execute('CREATE TABLE IF NOT EXISTS dividends (ticker text, date text, dividends float)')
conn.commit()

In [3]:
#Find sum of dividend for each stock
c.execute('SELECT ticker, sum(dividends * amount_of_shares) as total_div FROM dividends group by ticker')
results = c.fetchall() #List of tuples
for result in results:
    print(result) #Returns a tuple

('1830.HK', 4912.0)
('3988.HK', 2028.7520000000002)
('D05.SI', 564.0)
('H78.SI', 100.0)
('M44U.SI', 169.88400000000001)
('ME8U.SI', 333.2783)
('N2IU.SI', 64.8)
('O39.SI', 338.40000000000003)


In [28]:
c.execute('''
          SELECT ticker,
          SUM(
          CASE WHEN ticker = "1830.HK" OR ticker = "3988.HK" THEN 
            CASE WHEN (dividends * amount_of_shares * 0.02) < 30
            THEN ((dividends * amount_of_shares) - 30)
            ELSE (dividends * amount_of_shares * 0.98)
            END
          ELSE dividends * amount_of_shares
          END) AS total_div
          FROM dividends
          GROUP BY ticker
          ''')
results = c.fetchall() #List of tuples
for result in results:
    print(result) #Returns a tuple

('1830.HK', 4152.0)
('3988.HK', 1967.6826800000001)
('D05.SI', 564.0)
('H78.SI', 100.0)
('M44U.SI', 169.88400000000001)
('ME8U.SI', 333.2783)
('N2IU.SI', 64.8)
('O39.SI', 338.40000000000003)


In [6]:
#Refresh sqlite3 db
c.execute('DROP TABLE dividends')
conn.commit()

In [8]:
#Close the connection
conn.close()